In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('CH_NULL.csv', delimiter=';')
df_l1 = pd.read_csv('CH_L1.csv', delimiter=';')
df_l2 = pd.read_csv('CH_L2.csv', delimiter=';')
df_l3 = pd.read_csv('CH_L3.csv', delimiter=';')
df_l4 = pd.read_csv('CH_L4.csv', delimiter=';')
df_l5= pd.read_csv('CH_L5.csv', delimiter=';')
df_l6 = pd.read_csv('CH_L6.csv', delimiter=';')


/tmp/ipykernel_92807/349902543.py:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df_l6 = pd.read_csv('CH_L6.csv', delimiter=';')


In [3]:
# concatneate all dataframes
df_final = pd.concat([df_l1, df_l2, df_l3, df_l4, df_l5, df_l6], ignore_index=True)

In [4]:
# 1. Extraer U_Estilo de ItemName (formato: ESTILO/DESCRIPCION/TALLA/COLOR)
df['U_Estilo'] = df['ItemName'].str.split('/').str[0]


# 2. Asignar U_Genero y U_Categoria basado en el primer carácter
conditions = [
    df['U_Estilo'].str.startswith('F'),
    df['U_Estilo'].str.startswith('W'),
    df['U_Estilo'].str.startswith('C'),
    df['U_Estilo'].str.startswith('U')
]
choices = ['MACC', 'WFW', 'MFW', 'WACC']
df['U_Genero'] = np.select(conditions, choices, default='')
df['U_Categoria'] = df['U_Genero']  # Mismo valor que U_Genero

# 3. Asignar U_Segmento según reglas
df['U_Segmento'] = np.where(
    df['U_Genero'].isin(['WFW', 'MFW']),
    'FOOTWEAR',
    np.where(df['U_Genero'].isin(['MACC', 'WACC']), 'ACCESSORIES', '')
)

# 4. Seleccionar solo las columnas requeridas
columnas_finales = [
    'ItemName', 'U_Estilo', 'U_Genero', 'U_Categoria', 'U_Segmento','U_Descripcion',
    'U_Segmentacion_SK', 'U_Zone', 'U_Descrip_Color', 'ItemCode', 'Empresa'
]
df = df[columnas_finales]

# 5. Mostrar las primeras filas
df.head()

,ItemName,U_Estilo,U_Genero,U_Categoria,U_Segmento,U_Descripcion,U_Segmentacion_SK,U_Zone,U_Descrip_Color,ItemCode,Empresa
0,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,C31218,MFW,MFW,FOOTWEAR,NaN,NaN,NaN,NaN,201003812982267,SBO_00_DISRESALV
1,F12581/SHOE BAG/OS/DARK NAVY,F12581,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,203009730300167,SBO_00_DISRESALV
2,F12582/LOCKER BAG/OS/DARK NAVY,F12582,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,203009740300167,SBO_00_DISRESALV
3,F12583/CART BAG/OS/DARK NAVY,F12583,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,203009750300167,SBO_00_DISRESALV
4,U07433/ELIZABETH CROSSBODY/OS/BLACK,U07433,WACC,WACC,ACCESSORIES,NaN,NaN,NaN,NaN,103009633030167,SBO_00_DISRESALV


In [5]:
# Hacer merge controlando mejor las columnas
df_resultado = pd.merge(
    df.drop(columns=['U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color'],  # Elimina columnas que vamos a reemplazar
           errors='ignore'),  # Elimina columnas que vamos a reemplazar
    df_final[['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']],
    on='U_Estilo',
    how='left'
)

# Asegurar el orden de columnas
column_order = [
    'ItemName', 'ItemCode', 'Empresa', 'U_Estilo', 'U_Genero', 'U_Categoria',
    'U_Segmento', 'U_Descripcion', 'U_Descrip_Color', 'U_Segmentacion_SK', 'U_Zone'
]

# Reordenar y seleccionar solo las columnas deseadas
df_resultado = df_resultado[column_order]

# Verificar el resultado
df_resultado.head()

,ItemName,ItemCode,Empresa,U_Estilo,U_Genero,U_Categoria,U_Segmento,U_Descripcion,U_Descrip_Color,U_Segmentacion_SK,U_Zone
0,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,NaN,BRITISH T AN - IVOR,GRANDPRO,SPORT
1,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport
2,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport
3,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport
4,F12581/SHOE BAG/OS/DARK NAVY,203009730300167,SBO_00_DISRESALV,F12581,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN


In [6]:

df_resultado['U_Talla'] = df_resultado['ItemName'].str.split('/').str[2]

df_resultado
df_resultado.head()


,ItemName,ItemCode,Empresa,U_Estilo,U_Genero,U_Categoria,U_Segmento,U_Descripcion,U_Descrip_Color,U_Segmentacion_SK,U_Zone,U_Talla
0,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,NaN,BRITISH T AN - IVOR,GRANDPRO,SPORT,13
1,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport,13
2,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport,13
3,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport,13
4,F12581/SHOE BAG/OS/DARK NAVY,203009730300167,SBO_00_DISRESALV,F12581,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,OS


In [7]:
df_resultado.to_csv('resultado2.csv', index=False, sep=';')